# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [ ]:
import pandas as pd
import os
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from utils.plots import plot_features_time_series, contact_gsr_line_plot, plot_segment_violin
from utils.Tests import Tests
import statsmodels.formula.api as smf

In [ ]:
# Create new directory for notebook output
notebook_temp_dir = os.path.join(os.getcwd(), "temp", "3_statistics")

if not os.path.exists(notebook_temp_dir):
    os.makedirs(notebook_temp_dir)

In [ ]:
# File containing features for entire segments
segment_features_file = 'D:\\co2-study\\temp\\segment_features.csv'
segment_features = pd.read_csv(segment_features_file, index_col=0)

# File containing features for windows of data
windowed_features_file = 'D:\\co2-study\\temp\\windowed_features.csv'
windowed_features = pd.read_csv(windowed_features_file, index_col=0)

In [ ]:
# Filter the data for 'air' and 'co2' conditions with 'gas_inhalation' segment
gas_inhalation_segments = segment_features[(segment_features['Condition'].isin(['AIR', 'CO2'])) & (segment_features['Segment'] == 'gas_inhalation')]
gas_inhalation_windows = windowed_features[(windowed_features['Condition'].isin(['AIR', 'CO2'])) & (windowed_features['Segment'] == 'gas_inhalation')]

# Calculate mean combined pupil size using left and right pupils
gas_inhalation_segments['pupil_size_combined'] = (gas_inhalation_segments['VerboseData.Left.PupilDiameterMm_mean'] + gas_inhalation_segments['VerboseData.Right.PupilDiameterMm_mean']) / 2
gas_inhalation_windows['pupil_size_combined'] = (gas_inhalation_windows['VerboseData.Left.PupilDiameterMm_mean'] + gas_inhalation_windows['VerboseData.Right.PupilDiameterMm_mean']) / 2

# add window index for each participant/condition
window_index = pd.DataFrame({'window_index': gas_inhalation_windows.groupby(['participant_number', 'Condition']).cumcount()})
gas_inhalation_windows.insert(3, 'window_index', window_index['window_index'])

# GSR


In [ ]:
# Filter GSR data for segmented and windowed data
gsr_data_segments = gas_inhalation_segments.filter(regex=r'^(participant_number|Condition|Segment|Biopac_GSR_)').reset_index(drop=True)
gsr_air_segments = gsr_data_segments[gsr_data_segments['Condition']=='AIR']
gsr_co2_segments = gsr_data_segments[gsr_data_segments['Condition']=='CO2']

gsr_data_windows = gas_inhalation_windows.filter(regex=r'^(participant_number|Condition|Segment|window_index|Biopac_GSR_)').reset_index(drop=True)
gsr_air_windows = gsr_data_windows[gsr_data_windows['Condition']=='AIR']
gsr_co2_windows = gsr_data_windows[gsr_data_windows['Condition']=='CO2']

In [ ]:
# Run Shapiro-Wilk normality tests for both segmented and windowed GSR
Tests.normality_test_sw(gsr_air_segments['Biopac_GSR_mean'], 'AIR')
Tests.normality_test_sw(gsr_co2_segments['Biopac_GSR_mean'], 'CO2')

In [ ]:
# 
Tests.paired_t_test(gsr_air_segments['Biopac_GSR_mean'],gsr_co2_segments['Biopac_GSR_mean'])

In [ ]:
plot_segment_violin(gsr_data_segments, 'Biopac_GSR_mean', 'Mean GSR for Air and CO2 Conditions', 'Condition', 'GSR', 
                    notebook_temp_dir)

In [ ]:
plot_features_time_series(gas_inhalation_windows, 'Biopac_GSR_mean', 'GSR Mean Over Time', 'Time (minutes)', 'Mean GSR', os.path.join(notebook_temp_dir, "GSR_mean_over_time"))

In [ ]:
Tests.regression_tests(gsr_air_windows, gsr_co2_windows, gsr_data_windows, 'Biopac_GSR_mean')

# RSP


In [ ]:
# RSP
rsp_data_segments = gas_inhalation_segments.filter(regex=r'^(participant_number|Condition|Segment|window_index|Biopac_RSP_|RSP_)').reset_index(drop=True)
rsp_air_segments = rsp_data_segments[rsp_data_segments['Condition']=='AIR']
rsp_co2_segments = rsp_data_segments[rsp_data_segments['Condition']=='CO2']

rsp_data_windows = gas_inhalation_windows.filter(regex=r'^(participant_number|Condition|Segment|window_index|Biopac_RSP_|RSP_)').reset_index(drop=True)
rsp_air_windows = rsp_data_windows[rsp_data_windows['Condition']=='AIR']
rsp_co2_windows = rsp_data_windows[rsp_data_windows['Condition']=='CO2']

In [ ]:
# Run Shapiro-Wilk normality tests for both segmented and windowed GSR
Tests.normality_test_sw(rsp_air_segments['RSP_Rate_Mean'], 'AIR')
Tests.normality_test_sw(rsp_co2_segments['RSP_Rate_Mean'], 'CO2')

In [ ]:
Tests.paired_t_test(rsp_air_segments['RSP_Rate_Mean'],rsp_co2_segments['RSP_Rate_Mean'])

In [ ]:
plot_segment_violin(rsp_data_segments, 'RSP_Rate_Mean', 'Mean Respiratory Rate for Air and CO2 Conditions', 'Condition', 'Respiratory Rate', 
                    notebook_temp_dir)

In [ ]:
plot_features_time_series(gas_inhalation_windows, 'RSP_Rate_Mean', 'RSP Rate Mean Over Time', 'Time (minutes)', 'RSP Rate Mean', os.path.join(notebook_temp_dir, "RSP_mean_over_time"))

In [ ]:
Tests.regression_tests(rsp_air_windows, rsp_co2_windows, rsp_data_windows, 'RSP_Rate_Mean')

# HR


In [ ]:
# HR
#TODO: Using emteqPRO's HR because feature extraction when using entire segments sets HR features to nan per outlier rejection
hr_data_segments = gas_inhalation_segments.filter(regex=r'^(participant_number|Condition|Segment|PPG_|HRV_|HeartRate/Average_mean)').reset_index(drop=True)
hr_data_segments.rename(columns = {'HeartRate/Average_mean':'HeartRate_Average_mean'}, inplace = True)
hr_air_segments = hr_data_segments[hr_data_segments['Condition']=='AIR']
hr_co2_segments = hr_data_segments[hr_data_segments['Condition']=='CO2']

hr_data_windows = gas_inhalation_windows.filter(regex=r'^(participant_number|Condition|Segment|window_index|PPG_|HRV_|HeartRate/Average_mean)').reset_index(drop=True)
hr_data_windows.rename(columns = {'HeartRate/Average_mean':'HeartRate_Average_mean'}, inplace = True)
hr_air_windows = hr_data_windows[hr_data_windows['Condition']=='AIR']
hr_co2_windows = hr_data_windows[hr_data_windows['Condition']=='CO2']

In [ ]:
# Run Shapiro-Wilk normality tests for both segmented and windowed GSR
Tests.normality_test_sw(hr_air_segments['HeartRate_Average_mean'], 'AIR')
Tests.normality_test_sw(hr_co2_segments['HeartRate_Average_mean'], 'CO2')

In [ ]:
Tests.paired_t_test(hr_air_segments['HeartRate_Average_mean'],hr_co2_segments['HeartRate_Average_mean'])

In [ ]:
plot_segment_violin(hr_data_segments, 'HeartRate_Average_mean', 'Mean Heart Rate for Air and CO2 Conditions', 'Condition', 'Heart Rate', 
                    notebook_temp_dir)

In [ ]:
plot_features_time_series(gas_inhalation_windows, 'HeartRate/Average_mean', 'HR Rate Mean Over Time', 'Time (minutes)', 'HR Rate Mean (Emteq)', os.path.join(notebook_temp_dir, "HR_mean_over_time_EMTEQ"))

In [ ]:
plot_features_time_series(gas_inhalation_windows, 'PPG_Rate_Mean', 'HR Rate Mean Over Time', 'Time (minutes)', 'HR Rate Mean (PPG)', os.path.join(notebook_temp_dir, "GSR_mean_over_time_PPG"))

In [ ]:
Tests.regression_tests(hr_air_windows, hr_co2_windows, hr_data_windows, 'HeartRate_Average_mean')

# Pupil Size

In [ ]:
# Pupil size
pupil_data_segments = gas_inhalation_segments.filter(regex=r'^(participant_number|Condition|Segment|VerboseData.|pupil_)').reset_index(drop=True)
#pupil_data_segments['pupil_size_combined'] = (pupil_data_segments['VerboseData.Left.PupilDiameterMm_mean'] + pupil_data_segments['VerboseData.Right.PupilDiameterMm_mean']) / 2

pupil_air_segments = pupil_data_segments[pupil_data_segments['Condition']=='AIR']
pupil_co2_segments = pupil_data_segments[pupil_data_segments['Condition']=='CO2']

pupil_data_windows = windowed_features.filter(regex=r'^(participant_number|Condition|Segment|VerboseData.|pupil_)').reset_index(drop=True)
pupil_air_windows = pupil_data_windows[pupil_data_windows['Condition']=='AIR']
pupil_co2_windows = pupil_data_windows[pupil_data_windows['Condition']=='CO2']

In [ ]:
# Shapiro-Wilk test for normality
shapiro_test = stats.shapiro(pupil_air_segments['pupil_size_combined'])
print("Combined Pupil Size Mean - AIR:")
print("Shapiro-Wilk Test - Statistic:", shapiro_test.statistic)
print("Shapiro-Wilk Test - p-value:", shapiro_test.pvalue)

print("Combined Pupil Size Mean - CO2:")
shapiro_test = stats.shapiro(pupil_co2_segments['pupil_size_combined'])
print("Shapiro-Wilk Test - Statistic:", shapiro_test.statistic)
print("Shapiro-Wilk Test - p-value:", shapiro_test.pvalue)

# HR data is not normally disributed but due to sample size 50+, it is acceptable to run paired t test.

# Perform the paired t-test
t_statistic, p_value = stats.ttest_rel(pupil_air_segments['pupil_size_combined'], pupil_co2_segments['pupil_size_combined'])
print(f"T-statistic: {t_statistic:.3f}")
print(f"P-value: {p_value:.3f}")

In [ ]:
# Create the violin plot
sns.violinplot(x='Condition', y='pupil_size_combined', data=pupil_data_segments)

# Customize plot titles and labels
sns.set(style='whitegrid')
plt.title('Mean Pupil Size for Air and CO2 Conditions')
plt.xlabel('Condition')
plt.ylabel('Pupil Size Mean')

# Display the plot
plt.show()

In [ ]:
plot_features_time_series(gas_inhalation_windows, 'pupil_size_combined', 'Pupil Size Mean Over Time', 'Time windows', 'Pupil Size', os.path.join(notebook_temp_dir, "Pupil_size"))

# EMG Contact

In [ ]:
# Pupil size
contact_data_segments = gas_inhalation_segments.filter(regex=r'^(participant_number|Condition|Segment|Emg/Contact)').reset_index(drop=True)

contact_air_segments = contact_data_segments[contact_data_segments['Condition']=='AIR']
contact_co2_segments = contact_data_segments[contact_data_segments['Condition']=='CO2']

contact_data_windows = windowed_features.filter(regex=r'^(participant_number|Condition|Segment|Emg/Contact)').reset_index(drop=True)
contact_air_windows = contact_data_windows[contact_data_windows['Condition']=='AIR']
contact_co2_windows = contact_data_windows[contact_data_windows['Condition']=='CO2']

In [ ]:
# Shapiro-Wilk test for normality


In [ ]:
# Create the violin plot
sns.violinplot(x='Condition', y='Emg/Contact[RightOrbicularis]_mean', data=contact_data_segments)

# Customize plot titles and labels
sns.set(style='whitegrid')
plt.title('Mean Pupil Size for Air and CO2 Conditions')
plt.xlabel('Condition')
plt.ylabel('Pupil Size Mean')

# Display the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter the dataframe for 'AIR' and 'CO2' conditions
air_co2_data = contact_data_segments.copy()

# Melt the dataframe to convert separate columns for each muscle into a single column
melted_data = air_co2_data.melt(id_vars=['Condition', 'Segment'], value_vars=['Emg/Contact[RightOrbicularis]_mean', 'Emg/Contact[RightZygomaticus]_mean', 'Emg/Contact[RightFrontalis]_mean', 'Emg/Contact[CenterCorrugator]_mean', 'Emg/Contact[LeftFrontalis]_mean', 'Emg/Contact[LeftZygomaticus]_mean', 'Emg/Contact[LeftOrbicularis]_mean'], var_name='Muscle', value_name='Emg_Contact')

# Define the order of the muscles for plotting
muscle_order = ['RightOrbicularis', 'RightZygomaticus', 'RightFrontalis', 'CenterCorrugator', 'LeftFrontalis', 'LeftZygomaticus', 'LeftOrbicularis']

# Create the violin plot
plt.figure(figsize=(12, 8))
sns.violinplot(x='Condition', y='Emg_Contact', hue='Muscle', data=melted_data, scale='width', linewidth=0.5)


# Customize the plot
plt.title('EMG/Contact for Each Muscle in AIR and CO2 Conditions')
plt.xlabel('Condition')
plt.ylabel('EMG/Contact')
plt.legend(title='Muscle')

# Show the plot
plt.show()


In [ ]:
muscles = ['LeftOrbicularis','RightOrbicularis', 'LeftFrontalis', 'RightFrontalis', 'LeftZygomaticus', 'RightZygomaticus', 'CenterCorrugator']

for muscle in muscles:
    muscle_name = f'Emg/Contact[{muscle}]_mean'
    plot_title = f'{muscle} Mean Over Time'
    contact_gsr_line_plot(gas_inhalation_windows, muscle_name, plot_title, notebook_temp_dir)


# Correlation Matrix

In [ ]:
# Generate correlation matrix
# create a list of column names that contain "mean" or "Mean"
mean_columns = gas_inhalation_windows.columns[gas_inhalation_windows.columns.str.contains('_mean|_Mean')].tolist()

# select only the columns that contain "mean" or "Mean"
df = gas_inhalation_windows[mean_columns]
#df_mean = df_mean.dropna()



In [ ]:
import pandas as pd
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# compute the correlation matrix
corr_matrix = df.corr(method=lambda x, y: pearsonr(x, y)[0])

# compute the p-values for each correlation coefficient
p_values = df.corr(method=lambda x, y: pearsonr(x, y)[1])

# adjust the p-values using FDR control
reject, p_values_fdr = multipletests(p_values.values.flatten(), alpha=0.05, method='fdr_by')[:2]
p_values_fdr = pd.DataFrame(p_values_fdr.reshape(p_values.shape), index=p_values.index, columns=p_values.columns)

# use the adjusted p-values to filter the correlation matrix
corr_matrix_fdr = corr_matrix.where(p_values_fdr < 0.05)
p_values_fdr = p_values_fdr.where(p_values_fdr < 0.05)
corr_matrix_fdr.to_csv(os.path.join(notebook_temp_dir, "correlation_matrix_test_statistic.csv"))
p_values_fdr.to_csv(os.path.join(notebook_temp_dir, "correlation_matrix_p_values.csv"))


In [ ]:
import seaborn as sns
# select the subset of columns you want to plot
subset_cols = ['Biopac_GSR_mean', 'RSP_Rate_Mean']

# select the subset of the correlation matrix you want to plot
corr_matrix_fdr_subset = corr_matrix_fdr[subset_cols][subset_cols]

# create a boolean mask for rows where the index contains "Contact" or "Pupil"
#mask = corr_matrix_fdr_subset.index.str.contains("Contact|Pupil")
mask = corr_matrix_fdr_subset.index.str.contains("Contact")

# use boolean indexing to select the rows that meet the condition
corr_matrix_fdr_subset = corr_matrix_fdr_subset.loc[mask]

#corr_matrix_fdr_subset = corr_matrix_fdr_subset.fillna('')
# Create heatmap using seaborn
sns.heatmap(corr_matrix_fdr_subset, annot=True, cmap='coolwarm', square=False)
